# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ChampionsLeague2020-21").getOrCreate()

# Leer el archivo CSV
file_path = "dbfs:/FileStore/shared_uploads/uefa_2020.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Filtrar los partidos jugados en casa (en el estadio del equipo local)
df_home_games = df.filter(col("TEAM_HOME_SCORE").isNotNull())

# Agrupar por el nombre del equipo local (TEAM_NAME_HOME) y sumar los goles en casa (TEAM_HOME_SCORE)
df_team_goals = df_home_games.groupBy("TEAM_NAME_HOME").sum("TEAM_HOME_SCORE")

# Renombrar la columna de suma de goles para que sea más clara
df_team_goals = df_team_goals.withColumnRenamed("sum(TEAM_HOME_SCORE)", "HOME_GOALS")

# Ordenar por los goles en casa en orden descendente
df_team_goals = df_team_goals.orderBy(col("HOME_GOALS").desc())

# Seleccionar los 3 equipos con más goles en casa
df_top_teams = df_team_goals.limit(3)

# Mostrar el resultado final
df_top_teams.show()

+-----------------+----------+
|   TEAM_NAME_HOME|HOME_GOALS|
+-----------------+----------+
| Manchester City |        15|
|     Real Madrid |        14|
|   Bayern Munich |        13|
+-----------------+----------+



2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ChampionsLeague2021-22").getOrCreate()

# Leer el archivo CSV
file_path = "dbfs:/FileStore/shared_uploads/uefa_2021.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Columnas que necesitamos: POSSESSION_HOME, POSSESSION_AWAY , TEAM_NAME_HOME , TEAM_NAME_AWAY

#Juntar las columnas POSSESSION_HOME y POSSESSION_AWAY en POSSESION y las columnas TEAM_NAME_HOME y TEAM_NAME_AWAY en TEAM_NAME
df_combined = df.select(
    F.col("POSSESSION_HOME").alias("POSSESION"),
    F.col("TEAM_NAME_HOME").alias("TEAM_NAME")
).union(
    df.select(
        F.col("POSSESSION_AWAY").alias("POSSESION"),
        F.col("TEAM_NAME_AWAY").alias("TEAM_NAME")
    )
)

df_final = df_combined.groupBy("TEAM_NAME").agg(F.count("POSSESION").alias("POSSESION_WIN")).orderBy("POSSESION_WIN", ascending=False).limit(1)
df_final.display()


TEAM_NAME,POSSESION_WIN
Real Madrid,13


3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Crear una sesión de Spark
spark = SparkSession.builder.appName("UEFA2022-23").getOrCreate()

# Leer el archivo CSV (suponiendo que el CSV ya está cargado)
file_path = "dbfs:/FileStore/shared_uploads/uefa_2022.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Filtramos los equipos que ganaron más duelos pero perdieron el juego
df_filtered = df.filter(
    ((F.col("DUELS_WON_HOME") > F.col("DUELS_WON_AWAY")) & (F.col("TEAM_HOME_SCORE") < F.col("TEAM_AWAY_SCORE"))) |  # Local ganó los duelos pero perdió el juego
    ((F.col("DUELS_WON_AWAY") > F.col("DUELS_WON_HOME")) & (F.col("TEAM_AWAY_SCORE") < F.col("TEAM_HOME_SCORE")))   # Visitante ganó los duelos pero perdió el juego
)

# Seleccionamos las columnas necesarias y renombramos para los resultados
df_result = df_filtered.select(
    "STAGE",
    # Si el local ganó los duelos pero perdió el juego, entonces el local es el que perdió
    F.when(F.col("DUELS_WON_HOME") > F.col("DUELS_WON_AWAY"), F.col("TEAM_NAME_HOME"))
     .otherwise(F.col("TEAM_NAME_AWAY")).alias("TEAM_LOST")
)

# Mostramos el resultado final
df_result.display()


STAGE,TEAM_LOST
Group stage: Matchday 1,Chelsea
Group stage: Matchday 1,K�benhavn
Group stage: Matchday 1,Juventus
Group stage: Matchday 1,Maccabi Haifa
Group stage: Matchday 1,Rangers
Group stage: Matchday 1,Liverpool
Group stage: Matchday 1,Porto
Group stage: Matchday 1,Bayer Leverkusen
Group stage: Matchday 1,Viktoria Plze?
Group stage: Matchday 1,Marseille
